# IMPORT

In [64]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from math import ceil

# PLOTS

In [65]:
def plotBoxplot(data,name,k):
    quartiles = [[list()] * 5] * 16
    for ai in range(len(data)):
        a = data[ai]
        x = round(np.sqrt(len(a))) 
        y = ceil(len(a)/x)
        
        for di in range(len(a)):
            d = a[di]

            [q1,q2,q3] = np.percentile(d,[25, 50, 75])
            iqr = q3-q1
            whis = 1.5
            q0 = q1 - whis*iqr
            q4 = q3 + whis*iqr

            quartiles[ai][di] = [q0,q1,q2,q3,q4]
            plt.boxplot(d,whis=whis)

            plt.title(name[:3] + ' act ' + str(ai) + ' dev ' + str(di))
            plt.savefig('Plots/' + name + '/'+str(k)+'-' + name.lower() + '-' + str(ai) + '.jpg')
            plt.show()
            return [q0,q1,q2,q3,q4]
        
        plt.subplots_adjust(hspace=0.4,wspace=0.3)



    return quartiles

# READ FILE

In [92]:
dir_files = "../Dataset/"
            # rows = np.list(csv_reader).astype(np.float)

def readFiles():
    part = dict()
    d = 2
    i = 0
    
    file_name = dir_files + "part" + str(i) + "/part" + str(i) + "dev" + str(d) + ".csv"
    csv_file = open(file_name)
    csv_reader = csv.reader(csv_file, delimiter=',')
    rows = np.array(list(csv_reader)).astype(float)
    
    return rows

part = readFiles()

[[ 1.0000e+00 -1.8650e+00  9.3890e+00 ...  1.0134e+00  5.0589e+02
   1.0000e+00]
 [ 1.0000e+00 -1.7963e+00  9.3742e+00 ...  1.0000e+00  5.2542e+02
   1.0000e+00]
 [ 1.0000e+00 -1.8696e+00  9.3000e+00 ...  1.0156e+00  5.4495e+02
   1.0000e+00]
 ...
 [ 1.0000e+00 -2.1324e+00  9.3112e+00 ...  5.4343e-01  1.0379e+06
   1.0000e+00]
 [ 1.0000e+00 -2.1327e+00  9.3110e+00 ...  5.5234e-01  1.0380e+06
   1.0000e+00]
 [ 1.0000e+00 -2.1334e+00  9.2982e+00 ...  5.4120e-01  1.0380e+06
   1.0000e+00]]


# OUTLIERS

In [67]:
def outliersDensity(data,quartiles):
    density = [[list()] * 5] * 16
    for ai in range(len(data)):
        a = data[ai]

        for di in range(len(a)):
            d = a[di]
            q = quartiles[ai][di]

            outliers_minus = np.where(d < q[0])[0]
            outliers_plus = np.where(d > q[4])[0]

            outliers = np.concatenate((d[outliers_minus],d[outliers_plus]))

            density[ai][di] = len(outliers) / len(d) * 100
    
    return density
            

# BOXPLOT

In [106]:
def boxplot(part):
    acc = [None] * 16
    d = part
    for a in range(1,17):
        pos = np.where(d[:,11] == a)[0]

        acc[a-1] = np.sqrt(d[pos,1]**2 + d[pos,2]**2 + d[pos,3]**2)

    print(d[0][1],acc[0][0])

        # gyr[a-1] = np.sqrt(d[pos,4]**2 + d[pos,5]**2 + d[pos,6]**2)

        # mag[a-1] = np.sqrt(d[pos,7]**2 + d[pos,8]**2 + d[pos,9]**2)
        

        # density = outliersDensity(acc,quartiles_acc)

        # quartiles_gyr = plotBoxplot(gyr, "Gyroscope", k)
        
        # quartiles_mag = plotBoxplot(mag, "Magnemometer", k)


boxplot(part)

-1.865 9.9143400909995
